In [1]:
import numpy as np
from anytree import Node, RenderTree

In [ ]:
class LazyBranchAndBound:
    def __init__(self, A, k, initial_guess):
        # Validate inputs
        if not isinstance(k, int) or not isinstance(A, np.ndarray):
            raise ValueError("k must be an integer and A must be a numpy array")
        
        n = A.shape[0]
        if k > n:
            raise ValueError("k must be less than n")
        
        if A.shape[0] != A.shape[1]:
            raise ValueError("A must be a square matrix")
        
        if not np.all(A >= 0):
            raise ValueError("A must contain only non-negative values")
        
        if not np.allclose(A, A.T):
            raise ValueError("A must be symmetric")
        
        if not isinstance(initial_guess, (int, float)) or initial_guess < 0:
            raise ValueError("initial_guess must be a non-negative number")
        
        self.A = A
        self.k = k
        self.n = n
        self.best_guess = (initial_guess, [])
        self.root = Node(("root", 0, []), bound=float('inf'))

    def get_address(self, node):
        if node.is_root:
            return []
        return node.parent.name[2] + [node.name[0]]

    def _bound(self, node):
        """
        Computes the upper bound for a given node.
        For now, it returns +infinity as a dummy upper bound.

        :param node: The node for which to compute the upper bound.
        :return: The computed upper bound.
        """
        return float('inf')
    
    def _branch(self, node):
        # Create k children nodes
        for i in range(self.k):
            address = self.get_address(node) + [i]
            child = Node((i, node.depth, address), parent=node)
            child.bound = self._bound(child)

    def value(self, node):
        # Check if the node is an nth child of the root
        depth = node.depth
        if depth != self.n:
            raise ValueError("_value cannot be called on a set")
        
        # Use the address directly
        z = node.name[2]
        
        result = 0
        for i in range(self.n):
            for j in range(i+1,self.n):
                if z[i] != z[j]:
                    result += self.A[i][j]
        
        return result

    def _prune(self, node):
        # Recursively delete all children
        for child in node.children:
            self._prune(child)
        # Detach the node from its parent and delete it
        node.parent = None
        del node

    def _prune_ancestors(self, node): # This is not working as intended, trying to prune ancestors if they have no remaining children to save on memory.
        parent = node.parent
        while parent and not parent.is_root and len(parent.children) == 0:
            grandparent = parent.parent
            parent.parent = None
            del parent
            parent = grandparent
    
    def search(self):
        stack = [self.root]
        best_guess_node = None
        
        while stack:
            node = stack.pop()
            
            if node.depth < self.n:
                if node.bound >= self.best_guess[0]:
                    self._branch(node)
                    for child in node.children:
                        stack.append(child)
                else:
                    # Prune the node and its children
                    self._prune(node)
                    
            else:
                value = self.value(node)
                if value > self.best_guess[0]:
                    self.best_guess = (value, node.name[2])
                    print("Best guess:", self.best_guess)
                    
                    # Remove the old best_guess node from the stack and the tree
                    if best_guess_node:
                        if best_guess_node in stack:
                            stack.remove(best_guess_node)
                        self._prune(best_guess_node)
                    
                    # Update the best_guess_node
                    best_guess_node = node

                    # Add all siblings of the current node to the stack
                    if not node.is_root:
                        for sibling in node.parent.children:
                            if sibling != node:
                                stack.append(sibling)

                elif value < self.best_guess[0]:
                    
                    """
                    #Check all siblings first for pruning purposes.
                    if not node.is_root:
                        for sibling in node.parent.children:
                            if sibling != node:
                                value = self.value(sibling)
                                if value > self.best_guess[0]:
                                    self.best_guess = (value, sibling.name[2])
                                    print("Best guess:", self.best_guess)
                                    
                                    # Remove the old best_guess node from the stack and the tree
                                    if best_guess_node:
                                        if best_guess_node in stack:
                                            stack.remove(best_guess_node)
                                        self._prune(best_guess_node)
                                    
                                    # Update the best_guess_node
                                    best_guess_node = sibling 

                                    
                    # Backtrack and prune ancestors if they have no remaining children
                    self._prune_ancestors(node)
                    """

                    # Prune the node if its value is less than the current best guess
                    self._prune(node)

In [ ]:
# Test case
A = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
k = 2
initial_guess = 0

bb = LazyBranchAndBound(A, k, initial_guess)
bb.search()

# Render the tree, showing only the address component of each node
for pre, fill, node in RenderTree(bb.root):
    print("%s%s" % (pre, node.name[2]))

Best guess: (np.int64(1), [1, 1, 0])
Best guess: (np.int64(2), [1, 0, 1])
[]
├── [0]
│   ├── [0, 0]
│   └── [0, 1]
│       └── [0, 1, 0]
└── [1]
    ├── [1, 0]
    │   └── [1, 0, 1]
    └── [1, 1]


In [8]:
# Test case
A = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
k = 2
initial_guess = 0

bb = LazyBranchAndBound(A, k, initial_guess)
bb.search()

# Render the tree, showing only the address component of each node
for pre, fill, node in RenderTree(bb.root):
    print("%s%s" % (pre, node.name[2]))

Best guess: (np.int64(1), [1, 1, 0])
Best guess: (np.int64(2), [1, 0, 1])
[]
├── [0]
│   ├── [0, 0]
│   └── [0, 1]
│       └── [0, 1, 0]
└── [1]
    ├── [1, 0]
    │   └── [1, 0, 1]
    └── [1, 1]
